In [111]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint

In [112]:
def write_to_crawledPerfumes(data):
    client = MongoClient('localhost', 27017)
    Perfume_Database = client.Perfume_Database
    crawled_perfumes = Perfume_Database["Crawled_Perfumes"]
    crawled_perfumes.insert_many(data)

In [113]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame
import json
from selenium.webdriver.chrome.options import Options

Create a new profile path, and manually disable site data saving so fragrantica won't ask for cloudflare tests and also cookies.

In [114]:
def init_browser(): # MUST add settings to prevent chrome from saving site data
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=B:\Projects\Perfume-DataScrape\Chrome");
    
    return Browser("chrome", options=chrome_options, headless=False)

In [115]:
def filter_by_year(browser, year1, year2):
    browser.find_by_css('input[type="number"]')[1].fill(year2)
    browser.find_by_css('input[type="number"]')[0].fill(year1)
    
    time.sleep(0.5)

In [120]:
def filter_by_year_url(browser, year1, year2):
    url = f"https://www.fragrantica.com/search/?godina={year1}%3A{year2}"
    browser.visit(url)
    time.sleep(0.5)

In [121]:
def crawl_year(browser, year):
    soup = bs(browser.html, "html.parser")
    
    filter_by_year_url(browser, year, year)
    
#     MORE RESULTS
    for j in range(0,50):
        try:
    #         Wait for the more results button to load ( for 2 second at most)
            if browser.is_element_present_by_css('button[class="button"]', wait_time=2):
                loadMore_button = browser.find_by_css('button[class="button"]');
                if not loadMore_button._element.get_attribute('disabled'):
                    loadMore_button.click()
                    time.sleep(0.2)
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                else:
                    break  
            else:
                break
        except:
            break
        
#      Scrape PERFUME NAME AND LINKS
    soup = bs(browser.html, "html.parser")
    scraped_perfumes = []
    
        ####### GET THE LENGTH OF THE RESULTS #######
    block1 = soup.find("span", class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")

    for idx, perfume in enumerate(block1.find_all("a", href=True)):
        new_perfume = {}
        new_perfume['_id']  = year*10000 + idx
        new_perfume['name'] = perfume.get_text().replace('\n', '').strip(" ")
        new_perfume['link'] = perfume['href']
        scraped_perfumes.append(new_perfume)
        
    print(len(scraped_perfumes))
    
    return scraped_perfumes

In [117]:
browser = init_browser()

Exception ignored in: <function Service.__del__ at 0x0000025DAA80D300>
Traceback (most recent call last):
  File "B:\Software\Anaconda\Lib\site-packages\selenium\webdriver\common\service.py", line 185, in __del__
    self.stop()
  File "B:\Software\Anaconda\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "B:\Software\Anaconda\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "B:\Software\Anaconda\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "B:\Software\Anaconda\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "B:\Software\Anaconda\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
 

In [118]:
url = "https://www.fragrantica.com/search/"
# url = "https://www.fragrantica.com/perfume/Jean-Paul-Gaultier/Le-Male-Elixir-81642.html"
browser.visit(url)

In [124]:
for year in range(2023,2025):
    data = crawl_year(browser, year)
    
    try:
        if len(data)>0:
            write_to_crawledPerfumes(data)
    
    except errors.BulkWriteError as e:
        error = e
        print(e.details['writeErrors'])
        continue
        
    print(year)

910
2023
99
2024
